# Astropy: Tables


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Create tables</li>
<li>Access data in tables</li>
<li>Combining tables</li>
<li>Aggregation</li>
<li>Masking</li>
<li>Reading/writing</li>
</ul>

</div>

</section>


## Documentation

For more information about the features presented below, you can read the
[astropy.table](http://docs.astropy.org/en/stable/table/index.html) docs.

## Creating tables

In [1]:
from astropy.table import Table
import numpy as np

In [2]:
t1=Table()
t1['name']=['source 1','source 2','source 3']
t1['flux']=[1.2, 2.2, 3.1]

In [3]:
t1

name,flux
str8,float64
source 1,1.2
source 2,2.2
source 3,3.1


In [4]:
t1['size']=[1, 5, 4]
t1

name,flux,size
str8,float64,int32
source 1,1.2,1
source 2,2.2,5
source 3,3.1,4


In [5]:
t1['size']

1
5
4


In [6]:
np.array(t1['size'])

array([1, 5, 4])

In [7]:
t1['size'][0]

1

In [8]:
t1[0]

name,flux,size
str8,float64,int32
source 1,1.2,1


In [9]:
t1[0]['size']

1

## Units in tables

In [10]:
t1['size'].unit = 'cm'
t1

name,flux,size
,,cm
str8,float64,int32
source 1,1.2,1
source 2,2.2,5
source 3,3.1,4


Some unitful operations will then work:

In [11]:
t1['size'].to('m')

<Quantity [0.01, 0.05, 0.04] m>

However, you may run into unexpected behavior, so if you are planning on using table columns as Quantities, we recommend that you use the ``QTable`` class:

In [12]:
type(t1['size'])

astropy.table.column.Column

In [13]:
from astropy.table import QTable
qt1=QTable(t1)

In [14]:
type(qt1['size'])

astropy.units.quantity.Quantity


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<ol>
<li>Make a table that contains three columns: <code>spectral type</code>, <code>temperature</code>, and <code>radius</code>, and incude 5 rows with fake data (or real data if you like, for example from <a href="http://www.atlasoftheuniverse.com/startype.html">here</a>). Try including units on the columns that can have them.</li>
<li>Find the mean temperature and the maximum radius</li>
<li>Try and find out how to add and remove rows</li>
<li>Add a new column which gives the luminosity (using $L=4\pi R^2 \sigma T^4$)</li>
</ol>

</div>

</section>


In [15]:
import astropy.units as u
t2=QTable()

t2['Spectral_Type']=['O','B','A','F','G']
t2['Temperature']=([40000, 20000, 8500, 6500, 5700] * u.K)
t2['Radius']=([10, 5, 1.7, 1.3, 1] * u.R_sun)
t2

Spectral_Type,Temperature,Radius
,K,solRad
str1,float64,float64
O,40000.0,10.0
B,20000.0,5.0
A,8500.0,1.7
F,6500.0,1.3
G,5700.0,1.0


In [16]:
np.mean(t2['Temperature'])

<Quantity 16140. K>

In [17]:
np.max(t2['Radius'])

<Quantity 10. solRad>

In [18]:
t2['lifetime']=[10, 100, 1000, 3000, 10000]
t2['lifetime'].unit='Million_yrs'
t2

Spectral_Type,Temperature,Radius,lifetime
,K,solRad,Million_yrs
str1,float64,float64,float64
O,40000.0,10.0,10.0
B,20000.0,5.0,100.0
A,8500.0,1.7,1000.0
F,6500.0,1.3,3000.0
G,5700.0,1.0,10000.0


In [19]:
t2.add_row(vals=None)
t2

Spectral_Type,Temperature,Radius,lifetime
,K,solRad,Million_yrs
str1,float64,float64,float64
O,40000.0,10.0,10.0
B,20000.0,5.0,100.0
A,8500.0,1.7,1000.0
F,6500.0,1.3,3000.0
G,5700.0,1.0,10000.0
,0.0,0.0,0.0


In [20]:
t2.remove_row(5)
t2

Spectral_Type,Temperature,Radius,lifetime
,K,solRad,Million_yrs
str1,float64,float64,float64
O,40000.0,10.0,10.0
B,20000.0,5.0,100.0
A,8500.0,1.7,1000.0
F,6500.0,1.3,3000.0
G,5700.0,1.0,10000.0


In [21]:
from astropy.constants import sigma_sb
sigma_sb

<<class 'astropy.constants.codata2014.CODATA2014'> name='Stefan-Boltzmann constant' value=5.670367e-08 uncertainty=1.3e-13 unit='W / (K4 m2)' reference='CODATA 2014'>

In [23]:
t2['Luminosity']=(4*np.pi*sigma_sb*(t2['Radius']**2) * (t2['Temperature']**4)).to(u.L_sun)
t2

Spectral_Type,Temperature,Radius,lifetime,Luminosity
,K,solRad,Million_yrs,solLum
str1,float64,float64,float64,float64
O,40000.0,10.0,10.0,230639.1747035746
B,20000.0,5.0,100.0,3603.737104743353
A,8500.0,1.7,1000.0,13.59147703493795
F,6500.0,1.3,3000.0,2.7178985454389313
G,5700.0,1.0,10000.0,0.9510263120351983


## Iterating over tables

It is possible to iterate over rows or over columns. To iterate over rows, simply iterate over the table itself:

In [24]:
for row in t1:
    print(row)

  name   flux size
               cm 
-------- ---- ----
source 1  1.2    1
  name   flux size
               cm 
-------- ---- ----
source 2  2.2    5
  name   flux size
               cm 
-------- ---- ----
source 3  3.1    4


In [25]:
row

name,flux,size
,,cm
str8,float64,int32
source 3,3.1,4


Rows can act like dictionaries, so you can access specific columns from a row:

In [27]:
for row in t1:
    print(row['name'])

source 1
source 2
source 3


Iterating over columns is also easy:

In [28]:
t1.colnames

['name', 'flux', 'size']

In [32]:
for colname in t1.colnames:
    print(t1[colname])

  name  
--------
source 1
source 2
source 3
flux
----
 1.2
 2.2
 3.1
size
 cm 
----
   1
   5
   4


Accessing specific rows from a column object can also be done with the item notation:

In [34]:
for colname in t1.colnames:
    print(t1[colname][0])

source 1
1.2
1


## Joining tables

In [35]:
from astropy.table import join

In [36]:
t = Table()
t['name']=['source 1', 'source 3']
t['flux2']=[1,9]

In [37]:
t1

name,flux,size
,,cm
str8,float64,int32
source 1,1.2,1
source 2,2.2,5
source 3,3.1,4


In [38]:
t

name,flux2
str8,int32
source 1,1
source 3,9


In [41]:
t3 = join(t1,t, join_type='outer')
t3

name,flux,size,flux2
,,cm,
str8,float64,int32,int32
source 1,1.2,1,1
source 2,2.2,5,--
source 3,3.1,4,9


## Masked tables

In [43]:
t3['flux2'].mask

array([False,  True, False])

## Slicing

Tables can be sliced like Numpy arrays:

In [47]:
obs=Table()
obs['name']=['M31', 'M32', 'M31', 'M40', 'M50']
obs['mag']=[4, 6, 8, 7, 6]

In [48]:
obs

name,mag
str3,int32
M31,4
M32,6
M31,8
M40,7
M50,6


In [49]:
obs[0:3]

name,mag
str3,int32
M31,4
M32,6
M31,8


In [52]:
obs[obs['mag']>6]

name,mag
str3,int32
M31,8
M40,7


In [53]:
obs['mag', 'name']

mag,name
int32,str3
4,M31
6,M32
8,M31
7,M40
6,M50



<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>Starting from the <code>obs</code> table:</p>
<ol>
<li>Make a new table that shows every other row, starting with the second row? (that is, the second, fourth, sixth, etc. rows).</li>
<li>Make a new table the only contains rows where <code>name</code> is <code>M31</code></li>
</ol>

</div>

</section>


In [59]:
obs[1::2]

name,mag
str3,int32
M32,6
M40,7


In [60]:
obs[obs['name']=='M31']

name,mag
str3,int32
M31,4
M31,8


## Grouping and Aggregation

In [61]:
obs_by_name=obs.group_by('name')

In [63]:
obs_by_name.groups

<TableGroups indices=[0 2 3 4 5]>

In [64]:
for group in obs_by_name.groups:
    print(group)
    print('')

name mag
---- ---
 M31   4
 M31   8

name mag
---- ---
 M32   6

name mag
---- ---
 M40   7

name mag
---- ---
 M50   6



In [65]:
obs_by_name.groups.aggregate(np.mean)

name,mag
str3,float64
M31,6.0
M32,6.0
M40,7.0
M50,6.0


## Writing data

In [66]:
obs.write('obstable.fits')

In [72]:
obs.write('obstable.vot', format='votable')

In [73]:
!cat obstable.vot

<?xml version="1.0" encoding="utf-8"?>
<!-- Produced with astropy.io.votable version 3.0.2
     http://www.astropy.org/ -->
<VOTABLE version="1.3" xmlns="http://www.ivoa.net/xml/VOTable/v1.3" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://www.ivoa.net/xml/VOTable/v1.3">
 <RESOURCE type="results">
  <TABLE>
   <FIELD ID="name" arraysize="3" datatype="unicodeChar" name="name"/>
   <FIELD ID="mag" datatype="int" name="mag"/>
   <DATA>
    <TABLEDATA>
     <TR>
      <TD>M31</TD>
      <TD>4</TD>
     </TR>
     <TR>
      <TD>M32</TD>
      <TD>6</TD>
     </TR>
     <TR>
      <TD>M31</TD>
      <TD>8</TD>
     </TR>
     <TR>
      <TD>M40</TD>
      <TD>7</TD>
     </TR>
     <TR>
      <TD>M50</TD>
      <TD>6</TD>
     </TR>
    </TABLEDATA>
   </DATA>
  </TABLE>
 </RESOURCE>
</VOTABLE>


## Reading data

In [74]:
t4=Table.read(2mass.tbl, format='ascii.ipac')

SyntaxError: invalid syntax (<ipython-input-74-fcadc2776fc9>, line 1)


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>Using the <code>t4</code> table above:</p>
<ol>
<li>
<p>Make a plot that shows <code>j_m</code>-<code>h_m</code> on the x-axis, and <code>h_m</code>-<code>k_m</code> on the y-axis</p>
</li>
<li>
<p>Make a new table that contains the subset of rows where the <code>j_snr</code>, <code>h_snr</code>, and <code>k_snr</code> columns, which give the signal-to-noise-ratio in the J, H, and K band, are greater than 10, and try and show these points in red in the plot you just made.</p>
</li>
<li>
<p>Make a new table (based on the full table) that contains only the RA, Dec, and the <code>j_m</code>, <code>h_m</code> and <code>k_m</code> columns, then try and write out this catalog into a format that you can read into another software package. For example, try and write out the catalog into CSV format, then read it into a spreadsheet software package (e.g. Excel, Google Docs, Numbers, OpenOffice). You may run into an issue at this point - if so, take a look at https://github.com/astropy/astropy/issues/7357 to see how to fix it.</p>
</li>
</ol>

</div>

</section>
